In [1]:
from google.protobuf import text_format
from brainlit.viz.visualize import napari_viewer

from ffn.utils import bounding_box_pb2
from ffn.inference import storage
from ffn.inference import inference
from ffn.inference import inference_pb2
from ffn.inference import inference_flags

import numpy as np
import h5py
import os
from skimage import io
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

D:\Anaconda3\envs\brainlit\lib\site-packages\napari\__init__.py:44: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.6. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [3]:
base_dir = Path("D:/Study/Nuero Data Design/brainlit")
data_dir = base_dir / "benchmarking_datasets"
im_dir = data_dir / "Images"
mask_dir = base_dir / "benchmarking_masks"
gfp_files = list(im_dir.glob("**/*.tif"))
save_path = "C:/Users/Tanch/ffn/brainlit/images"
test_path = save_path + "/test"
validation_path = save_path + "/validation"
if not os.path.exists(test_path):
    os.makedirs(test_path)
if not os.path.exists(validation_path):
    os.makedirs(validation_path)

for im_num, im_path in enumerate(gfp_files):
    file_name = im_path.parts[-1][:-8]
    print(file_name)
    if file_name[:10] != "validation":
        print("it's test img")
        im = io.imread(im_path, plugin="tifffile")
        im = np.swapaxes(im, 0, 2)
        mask_file = file_name + "_mask.tif"
        mask_path = mask_dir / mask_file
        mask = io.imread(mask_path, plugin="tifffile")
        test_file = test_path + "/" + file_name + ".h5"
        with h5py.File(test_file, 'w') as f:
            f.create_dataset('raw', data = im, compression = 'gzip') 
            f.create_dataset('mask', data = mask, compression = 'gzip')
    else:
        print("it's validation img")
        im = io.imread(im_path, plugin="tifffile")
        im = np.swapaxes(im, 0, 2)
        mask_file = file_name + "_mask.tif"
        mask_path = mask_dir / mask_file
        mask = io.imread(mask_path, plugin="tifffile")
        val_file = validation_path + "/" + file_name + ".h5"
        with h5py.File(val_file, 'w') as f:
            f.create_dataset('raw', data = im, compression = 'gzip') 
            f.create_dataset('mask', data = mask, compression = 'gzip')

test_1
it's test img
test_10
it's test img
test_11
it's test img
test_12
it's test img
test_13
it's test img
test_14
it's test img
test_15
it's test img
test_16
it's test img
test_17
it's test img
test_18
it's test img
test_19
it's test img
test_2
it's test img
test_20
it's test img
test_21
it's test img
test_22
it's test img
test_23
it's test img
test_24
it's test img
test_25
it's test img
test_3
it's test img
test_4
it's test img
test_5
it's test img
test_6
it's test img
test_7
it's test img
test_8
it's test img
test_9
it's test img
validation_1
it's validation img
validation_10
it's validation img
validation_11
it's validation img
validation_12
it's validation img
validation_13
it's validation img
validation_14
it's validation img
validation_15
it's validation img
validation_16
it's validation img
validation_17
it's validation img
validation_18
it's validation img
validation_19
it's validation img
validation_2
it's validation img
validation_20
it's validation img
validation_21
it's 

In [6]:
result_path = "C:/Users/Tanch/ffn/brainlit/results"
for i in range(25):
    i += 1
    test_path = result_path + "/test_" + str(i)
    if not os.path.exists(test_path):
        os.makedirs(test_path)
    for j in range(25):
        j += 1
        val_path = test_path + "/validation_" + str(j)
        if not os.path.exists(val_path):
            os.makedirs(val_path)
        val_path += "/0"
        if not os.path.exists(val_path):
            os.makedirs(val_path)
test_path = result_path + "/10_images"
for j in range(25):
    j += 1
    val_path = test_path + "/validation_" + str(j)
    if not os.path.exists(val_path):
        os.makedirs(val_path)
    val_path += "/0"
    if not os.path.exists(val_path):
        os.makedirs(val_path)

In [4]:
napari_viewer(im, labels = mask)

In [2]:
napari_view = True
for im_num in range(25):
    im_num += 1
    config = '''image {
      hdf5: "brainlit/images/validation/validation_%d.h5:raw"
    }
    image_mean: 64
    image_stddev: 33
    checkpoint_interval: 1800
    seed_policy: "PolicyPeaks"
    model_checkpoint_path: "brainlit/models/test_1/model.ckpt-10000"
    model_name: "convstack_3d.ConvStack3DFFNModel"
    model_args: "{\\"depth\\": 6, \\"fov_size\\": [11, 11, 11], \\"deltas\\": [4, 4, 4]}"
    segmentation_output_dir: "brainlit/results/test_1/validation_%d"
    inference_options {
      init_activation: 0.95
      pad_value: 0.05
      move_threshold: 0.9
      min_boundary_dist { x: 1 y: 1 z: 1}
      segment_threshold: 0.6
      min_segment_size: 1000
    }''' % (im_num, im_num)
    req = inference_pb2.InferenceRequest()
    _ = text_format.Parse(config, req)
    
    runner = inference.Runner()
    runner.start(req)
    canvas, alignment = runner.make_canvas((0, 0, 0), (330, 330, 100))
    
    bbox = bounding_box_pb2.BoundingBox()
    bounding_box = 'start { x:0 y:0 z:0 } size { x:330 y:330 z:100 }'
    text_format.Parse(bounding_box, bbox)
    
    runner.run((bbox.start.z, bbox.start.y, bbox.start.x), (bbox.size.z, bbox.size.y, bbox.size.x))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Please use `layer.__call__` method instead.


INFO:tensorflow:Restoring parameters from brainlit/models/test_1/model.ckpt-10000


...

INFO:tensorflow:Restoring parameters from brainlit/models/test_1/model.ckpt-10000


INFO:tensorflow:Restoring parameters from brainlit/models/test_1/model.ckpt-10000


INFO:tensorflow:Restoring parameters from brainlit/models/test_1/model.ckpt-10000


INFO:tensorflow:Restoring parameters from brainlit/models/test_1/model.ckpt-10000


INFO:tensorflow:Restoring parameters from brainlit/models/test_1/model.ckpt-10000


In [4]:
for im_num in range(25):
    im_num += 1
    seg, _ = storage.load_segmentation('brainlit/results/test_1/validation_%d' % im_num, (0, 0, 0))
    seg2 = seg
    seg2 = np.int64(seg2 > 0)
    
    filename = "brainlit/images/validation/validation_%d.h5" % im_num
    with h5py.File(filename, "r") as f:
        # List all groups
        print("Keys: %s" % f.keys())

        # Get the data
        data = list(f["raw"])
        img = np.array(data)
        data = list(f["mask"])
        mask = np.array(data)

    print(f"im_num = {im_num}")
    print(f"The number of labels: {sum(sum(sum(seg2)))}")    
    napari_viewer(img, labels=seg2)

Keys: <KeysViewHDF5 ['mask', 'raw']>
im_num = 1
The number of labels: 2839
Keys: <KeysViewHDF5 ['mask', 'raw']>
im_num = 2
The number of labels: 0
Keys: <KeysViewHDF5 ['mask', 'raw']>
im_num = 3
The number of labels: 0
Keys: <KeysViewHDF5 ['mask', 'raw']>
im_num = 4
The number of labels: 0
Keys: <KeysViewHDF5 ['mask', 'raw']>
im_num = 5
The number of labels: 0
Keys: <KeysViewHDF5 ['mask', 'raw']>
im_num = 6
The number of labels: 0
Keys: <KeysViewHDF5 ['mask', 'raw']>
im_num = 7
The number of labels: 0
Keys: <KeysViewHDF5 ['mask', 'raw']>
im_num = 8
The number of labels: 0
Keys: <KeysViewHDF5 ['mask', 'raw']>
im_num = 9
The number of labels: 3392
Keys: <KeysViewHDF5 ['mask', 'raw']>
im_num = 10
The number of labels: 0
Keys: <KeysViewHDF5 ['mask', 'raw']>
im_num = 11
The number of labels: 0
Keys: <KeysViewHDF5 ['mask', 'raw']>
im_num = 12
The number of labels: 0
Keys: <KeysViewHDF5 ['mask', 'raw']>
im_num = 13
The number of labels: 0
Keys: <KeysViewHDF5 ['mask', 'raw']>
im_num = 14
The n